In [ ]:
import os
import datetime as dt
import time
import pytz
import pandas as pd

from uce_resources import get_site_id, get_mms_data

In [ ]:
from settings.sites import ceg as sites_list

target_years = [2021]
target_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

target_folder = 'data/enercast_real_data/'
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

sites_list = ['Vasylivka']
sites_data = dict.fromkeys(sites_list)
print(sites_data)

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

In [ ]:
with engine.connect() as connection:
        
    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)
        mms=pd.DataFrame()
        for target_year in target_years:
            for target_month in target_months:

                mms_data, site_data['mms_version'] = get_mms_data(site_data['site_id'], 
                                                                target_year, target_month, 
                                                                connection, metadata.tables['mms_data'], include_prev=False)
                mms_data.index.name = 'DateTime'
                mms_data.columns = ['PowerWatt']
                mms_data['PowerWatt'] = mms_data['PowerWatt'] * 1000
                mms_data.index = mms_data.index - dt.timedelta(minutes=30)
                mms_data.index = mms_data.index.tz_localize(pytz.utc)
                mms_data.index = mms_data.index.strftime("%Y-%m-%d %H:%M%z")
                # mms_data.to_csv(target_folder + f'{site}_{target_year}_{target_month}_real_data_for_enercast.csv')
                print(f'Data for {site}: {target_year} {target_month} prepared!')
                mms=mms.append(mms_data)
                
        
        mms.to_csv(target_folder + f'{site}.csv')

